In [1]:
from swarm import Swarm, Agent
from swarm.types import Result
from dotenv import load_dotenv
import os

load_dotenv()

# Define the instructions for the Supply Agent
supply_agent_instructions = """
You are a Supply Agent responsible for processing surplus and in demand items for a food sharing network.

You will answer user queries about what items are available at a given location, or what items are needed at a given location.
The user will tell you what the location is and what items they have or need.
1. Parse the string into individual item categories such as fruits, vegetables, meat, etc. Use this for groups.
    - For example, if the user says they need poultry, this should be parsed into the category "meat".
    - If the user says they have strawberries and apples, this should be parsed into the category "fruits".
2. Use the food_mapping to map each item to its category, this should be empty if the type is demand.
3. Transfer the structured data to the Logistics Agent.

When calling save_items, make sure to always include the groups parameter.
"""


In [2]:

logistics_agent = Agent(name="Logistics Agent")

def save_items(context_variables, location_name: str, type: str, groups: list[str], food_mapping: dict = {}):
    """
    Processes and saves surplus or demand data for a given location.
    
    Args:
        location_name (str): The name of the location sending the surplus or demand.
        type (str): The type of data being sent; either "supply" for surplus or "demand" for deficit.
        groups (list of str): List of item categories being sent (e.g., ["fruits", "vegetables", "meat"]). Do not include the item names here. Instead categorize the items in the item names.
        food_mapping (dict, optional): A dictionary mapping each category to its items
                                        (e.g., {"fruits": ["strawberry", "apple"], "vegetables": ["carrot"], "meat": ["beef"]}).
                                        Required if type is "supply"; should be empty if type is "demand".
    
    Returns:
        Result or str: If type is "supply", returns a Result object to hand off to the Logistics Agent.
                      If type is "demand", returns a confirmation string.
    """
    print(groups)
    # Validate type
    if type not in ["supply", "demand"]:
        return "Invalid type provided. Must be either 'supply' or 'demand'."
    
    # Validate food_mapping based on type
    if type == "supply" and not food_mapping:
        return "food_mapping must be provided for type 'supply'."
    if type == "demand" and food_mapping:
        return "food_mapping should be empty for type 'demand'."
    
    # Initialize location in context_variables if not present
    if 'locations' not in context_variables:
        context_variables['locations'] = {}
    
    if location_name not in context_variables['locations']:
        context_variables['locations'][location_name] = {}
        
    # Now, based on the type, we update the context variables accordingly
    if type == "supply":
        if "surplus" in context_variables['locations'][location_name]:
            context_variables['locations'][location_name]["surplus"].extend(groups)
        else:
            context_variables['locations'][location_name]["surplus"] = groups

        if "surplus_mapping" in context_variables['locations'][location_name]:
            context_variables['locations'][location_name]["surplus_mapping"].update(food_mapping)
        else:
            context_variables['locations'][location_name]["surplus_mapping"] = food_mapping
    
    if type == "demand":
        if "demand" in context_variables['locations'][location_name]:
            context_variables['locations'][location_name]["demand"].extend(groups)
        else:
            context_variables['locations'][location_name]["demand"] = groups
        
    return Result(
        value=f"Parsed {type} data for Location {location_name}: {groups}",
        context_variables=context_variables,
        agent=logistics_agent
    )

In [3]:
# Create the Supply Agent with instructions and functions
supply_agent = Agent(
    name="Supply Agent",
    instructions=supply_agent_instructions,
    functions=[save_items]
)


In [4]:

# Initialize the Swarm client
client = Swarm()
# Simulate user messages indicating surplus
messages = [
    {"role": "user", "content": "Store: Test grocery store. Surplus items are: strawberries, apples, carrots, beef"}
]

# Define the function call context
def run_supply_agent():
    # Run the Supply Agent with the provided messages and additional arguments
    response = client.run(
        agent=supply_agent,
        messages=messages,
        context_variables={"locations": {}}
    )
    return response

# Execute the Supply Agent
response = run_supply_agent()

# Output the response
print("Final Agent:", response.agent.name)
print("Context Variables:", response.context_variables)
print("\nMessages:")
for msg in response.messages:
    print(f"{msg['role']}: {msg['content']}")

['fruits', 'vegetables', 'meat']
Final Agent: Logistics Agent
Context Variables: {'locations': {'Test grocery store': {'surplus': ['fruits', 'vegetables', 'meat'], 'surplus_mapping': {'fruits': ['strawberries', 'apples'], 'vegetables': ['carrots'], 'meat': ['beef']}}}}

Messages:
assistant: None
tool: Parsed supply data for Location Test grocery store: ['fruits', 'vegetables', 'meat']
assistant: The surplus items at Test grocery store have been categorized successfully into fruits (strawberries, apples), vegetables (carrots), and meat (beef). How would you like to proceed with these items?


In [5]:
# Simulate user messages indicating demand
demand_messages = [
    {"role": "user", "content": "Soup Kitchen, needs some beef"}
]

# Define the function call context for demand
def run_supply_agent():
    # Run the Supply Agent with the provided demand messages and additional arguments
    response = client.run(
        agent=supply_agent,
        messages=demand_messages,
        context_variables={"locations": {}}
    )
    return response

# Execute the Supply Agent for demand
demand_response = run_supply_agent()

# Output the response for demand
print("Final Agent (Demand):", demand_response.agent.name)
print("Context Variables (Demand):", demand_response.context_variables)
print("\nMessages (Demand):")
for msg in demand_response.messages:
    print(f"{msg['role']}: {msg['content']}")


['meat']
Final Agent (Demand): Logistics Agent
Context Variables (Demand): {'locations': {'Soup Kitchen': {'demand': ['meat']}}}

Messages (Demand):
assistant: None
tool: Parsed demand data for Location Soup Kitchen: ['meat']
assistant: The demand for beef has been recorded for the Soup Kitchen. Is there anything else you require assistance with?
